切换transformers版本 pip install transformers==4.30.2

In [1]:
# from modelscope.utils.hf_util import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
import torch
import json
from tqdm import tqdm
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("/mnt/workspace/.cache/modelscope/hub/ZhipuAI/chatglm3-6b",
                                          trust_remote_code=True)
device = torch.device('cuda:0')  # the device to load the model onto
model = AutoModelForCausalLM.from_pretrained("/mnt/workspace/.cache/modelscope/hub/ZhipuAI/chatglm3-6b",
                                             device_map={"": device},
                                             torch_dtype="auto",
                                             trust_remote_code=True)
model.to(device)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    # do_sample=True,
    temperature=0.1,
    top_p=0.95,
    top_k=40
)
hf = HuggingFacePipeline(pipeline=pipe)

2025-02-07 16:18:43.041360: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-07 16:18:43.316965: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-07 16:18:44.159111: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

The model 'ChatGLMForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'Rober

### 实验组

In [ ]:
# 读取 JSON 文件
file_path = 'aikps_output.json'  # 替换为你的文件路径

# 打开并读取 JSON 文件
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 初始化一个空列表，用于存储结果
results = []

# 使用 tqdm 创建进度条
for item in tqdm(data, desc="处理进度"):
    question = item[0]
    context = item[1][0]
    template ="""### 这是一道小学三年级人工智能学科的问题，请使用以下上下文来增强你的回答。
        {context}
        ### 问题: {question}
        ### 以{{"答案": ""}}的JSON格式返回最终结果。
    """
    prompt = PromptTemplate.from_template(template)
    # 假设 hf 是已经定义好的处理函数
    chain = prompt | hf
    # 修改传递给chain.invoke的参数，确保变量名和模板中的一致
    result = chain.invoke({"context": context, "question": question})

    # 将 context、question 和答案添加到结果列表中
    results.append([context, question, result])

# 将结果列表转换为 DataFrame
df = pd.DataFrame(results, columns=['Context', 'Question', 'Answer'])

# 将 DataFrame 写入新的 Excel 文件
excel_path = 'GLM-实验组.xlsx'  # 替换为你想要保存的 Excel 文件路径
df.to_excel(excel_path, index=False)

In [4]:
question = "人工智能科学家有谁？"
context = "'3.长得像人类的机器人才叫人工智能。(______)'"
template ="""### 这是一道小学三年级人工智能学科的问题，请使用以下上下文来增强你的回答。
    {context}
    ### 问题: {question}
    ### 以{{"答案": ""}}的JSON格式返回最终结果。
"""
prompt = PromptTemplate.from_template(template)
# 假设 hf 是已经定义好的处理函数
chain = prompt | hf
# 修改传递给chain.invoke的参数，确保变量名和模板中的一致
result = chain.invoke({"context": context, "question": question})
print(result)

### 这是一道小学三年级人工智能学科的问题，请使用以下上下文来增强你的回答。
    '3.长得像人类的机器人才叫人工智能。(______)'
    ### 问题: 人工智能科学家有谁？
    ### 以{"答案": ""}的JSON格式返回最终结果。
    {"答案": "人工智能科学家有Alan Turing、John McCarthy、Marvin Minsky、Norbert Wiener、Claude Shannon、Herbert Kauden"}


### 对照组

In [ ]:
# 读取 JSON 文件
file_path = 'aikps_output.json'  # 替换为你的文件路径

# 打开并读取 JSON 文件
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 初始化一个空列表，用于存储结果
results = []

# 使用 tqdm 创建进度条
for item in tqdm(data, desc="处理进度"):
    question = item[0]
    template ="""### 这是一道小学三年级人工智能学科的问题，请给出你的答案。
        ### 问题: {question}
        ### 以{{"答案": ""}}的JSON格式返回最终结果。
    """
    prompt = PromptTemplate.from_template(template)
    # 假设 hf 是已经定义好的处理函数
    chain = prompt | hf
    # 修改传递给chain.invoke的参数，确保变量名和模板中的一致
    result = chain.invoke({"question": question})

    # 将 context、question 和答案添加到结果列表中
    results.append([question, result])

# 将结果列表转换为 DataFrame
df = pd.DataFrame(results, columns=['Question', 'Answer'])

# 将 DataFrame 写入新的 Excel 文件
excel_path = 'GLM-对照组.xlsx'  # 替换为你想要保存的 Excel 文件路径
df.to_excel(excel_path, index=False)

In [3]:
question = "人工智能科学家有谁？"
template ="""### 这是一道小学三年级人工智能学科的问题，请给出你的答案。
    ### 问题: {question}
    ### 以{{"答案": ""}}的JSON格式返回最终结果。
"""
prompt = PromptTemplate.from_template(template)
# 假设 hf 是已经定义好的处理函数
chain = prompt | hf
# 修改传递给chain.invoke的参数，确保变量名和模板中的一致
result = chain.invoke({"question": question})
print(result)

### 这是一道小学三年级人工智能学科的问题，请给出你的答案。
    ### 问题: 人工智能科学家有谁？
    ### 以{"答案": ""}的JSON格式返回最终结果。
    
    请注意，由于我是一个人工智能助手，我可能对某些人物的了解可能有限。我会尽我所能提供有关人工智能领域的一些著名科学家。
    
    根据我的知识，以下是一些著名的人工智能科学家：
    
    - 艾伦·图灵：英国数学家、逻辑学家、计算机科学家，被认为是人工智能领域的奠基人之一。
    - 约翰·麦卡锡：美国计算机科学家、人工智能专家，是人工智能领域的先驱之一。
    - 克劳德·香农：美国数学家、逻辑学家、计算机科学家，是信息论和人工智能领域的奠基人之一。
    - 约瑟夫·Weizenbaum：美国计算机科学家，是人工智能领域的先驱之一，被誉为“人工智能之父”。
    -  Marvin Minsky：美国计算机科学家、人工智能专家，是人工智能领域的奠基人之一。
    - 约翰·霍普金斯：美国计算机科学家、人工智能专家，是人工智能领域的奠基人之一。
    
    这些科学家为人工智能领域的发展做出了巨大贡献。希望这些信息对您有所帮助！
    
    {"答案": "艾伦·图灵、约翰·麦卡锡、克劳德·香农、约瑟夫·Weizenbaum、Marvin Minsky、约翰·霍普金斯"}
